In [1]:
# ported from https://alachua.ihepa.ufl.edu:9999/tree/raid8/bockjoo/optimize/remote/Run2/Submit/limits/HZZ4l_Combination/CreateDatacards/CMSSW_6_1_1/src/HCG/makeBands_hzz4l.cc

from ROOT import *
import ROOT
import os
%jsroot on
#ROOT.gSystem.AddIncludePath("-I$ROOFITSYS/include/")
#ROOT.gSystem.AddIncludePath("-Iinclude/")
ROOT.gSystem.Load("/cvmfs/sft.cern.ch/lcg/releases/LCG_85swan3/vdt/0.3.6/x86_64-slc6-gcc49-opt/lib/libvdt.so")
#ROOT.gSystem.Load("../../lib/libHiggsAnalysisCombinedLimit.so")
ROOT.gSystem.Load("/softraid/bockjoo/combine_LCG85_swan3/HiggsAnalysis/CombinedLimit/lib/libHiggsAnalysisCombinedLimit.so")
#ROOT.gSystem.Load("include/HiggsCSandWidth_cc.so")
#ROOT.gSystem.Load("include/HiggsCSandWidthSM4_cc.so")
ROOT.gSystem.Load("/softraid/bockjoo/combine_LCG85_swan3/HiggsAnalysis/CombinedLimit/HiggsCSandWidth/include/HiggsCSandWidth_cc.so")
ROOT.gSystem.Load("/softraid/bockjoo/combine_LCG85_swan3/HiggsAnalysis/CombinedLimit/HiggsCSandWidth/include/HiggsCSandWidthSM4_cc.so")
# Setup python 
# import CMS tdrStyle
from tdrStyle import *
setTDRStyle()

# import some more python modules
import sys,glob
from array import array
import string
from scipy.special import erf
import math
#from bandUtils import *

Welcome to JupyROOT 6.07/07


/cvmfs/sft.cern.ch/lcg/views/LCG_85swan3/x86_64-slc6-gcc49-opt/lib/ROOT.py:417: UserWarning: "from ROOT import *" is not supported under IPython
  warnings.warn( '"from ROOT import *" is not supported under IPython' )


In [2]:
import pdefs
from plots import *
from bandUtils import *

In [3]:
prefix = ""
nfcbands = 4
fcbandnam = ["68", "95", "99", "9973" ]
fcbandwid = [ 0.68, 0.95, 0.99, 0.9973 ]

In [4]:

def cleanCLs(graph):
    foundBad = False
    while (foundBad) :
        foundBad = False
        for i in xrange(n) : # for (int i = 0, n = graph.GetN(); i < n; ++i):
            if (graph.GetX()[i] == 0): 
                graph.RemovePoint(i)
                foundBad = True
                break

In [5]:
def doIt(bands, name, rootfile):
    #import bandUtils
    print " name " , name , " access rootfile " , rootfile , " result = " , ROOT.gSystem.AccessPathName(rootfile)
    #//if (/*error=*/gSystem.AccessPathName(rootfile)) print " if " << std::endl
    if (ROOT.gSystem.AccessPathName(rootfile)) : return
    zero_is_valid = ("pval" in name or "sig" in name or "smcls" in name or "ml_" in name or "mlz_" in name)
    do_bands_asimov = "pval" in name
    use_precomputed_quantiles = "cls" in name or "ml_" in name or "mlz_" in name
    do_bands_95 = !("ml_" in name or "mlz_" in name)
    #//if ("pval" in name and name.Contains("comb")) do_bands_cputime = True
    makeBands(bands, name, rootfile, 0)
    print "Trying to do " , name
    if name.startswith("ml") : # if (name.index("ml") == 0):
        if (bands.Get(name+"_median") == 0):  print "Missing median??" ; return;  
        obs =  bands.Get(name+"_median").Clone()
        if (obs == 0) : return
        bands.Delete(name+"_obs;*")
        bands.Delete(name+"_median;*")
        obs.SetName(name+"_obs")
        bands.WriteTObject(obs, name+"_obs")
        printLineAErr(bands, name+"_obs", "results/"+prefix+name+".txt")
    else:
        #print "DEBUG results/"+prefix+name+".txt"
        printBand(bands, name, "results/"+prefix+name+".txt", False)
        if ("PLPE" in rootfile):
            printLine(bands, name+"_asimov", "results/"+prefix+name+"_expected.txt", False)
        
    
    zero_is_valid = False
    use_precomputed_quantiles = False
    do_bands_95 = True
    do_bands_asimov = False
    do_bands_cputime = False; do_bands_realtime = False


In [6]:
def doFc(bands, name, rootfile):
    if (ROOT.gSystem.AccessPathName(rootfile)) : return
    rfile = ROOT.TFile.Open(rootfile)
    for i in xrange(nfcbands) : # for (int i = 0; i < nfcbands; ++i):
        print "Make " , name
        fcb = theFcBelt(rfile, 1, 0, Observed, fcbandwid[i])
        if (fcb == 0) : continue
        fcb.SetName(name+"_"+fcbandnam[i])
        bands.WriteTObject(fcb)
    
    printFcBand(bands, name, name+".txt", nfcbands, fcbandnam)


In [7]:
def importCloned(bands, oldfile, oldname, nwhat, what,  nwho, who):
    oldBands = ROOT.TFile.Open(oldfile)
    for i in xrange(-2,nwho) : # for (int i = -2; i < nwho; ++i):        
        if i == -1: whoI = "comb"; who0 = "comb"
        elif i == -2: whoI = "hzz2l2q"; who0 = "hzz2l2q"
        else : whoI = who[i]; who0 = who[i]
        
        if (oldname == "lp11" and whoI == "hww") : whoI = "hwwc"
        for j in xrange(nwhat) : # for (int j = 0; j < nwhat; ++j):
            name  ="%s_%s" % (what[j], who0)
            nameI = "%s_%s" % (what[j], whoI)
            oldg_obs    =  oldBands.Get(nameI+"_obs")
            oldg_median =  oldBands.Get(nameI+"_median")
            oldg_medi95 =  oldBands.Get(nameI+"_median_95")
            if (oldg_obs):
                cleanCLs(oldg_obs)
                bands.WriteTObject(oldg_obs.Clone(name+"_"+oldname+"_obs"), name+"_"+oldname+"_obs")
                print "Imported " , oldname , " result " , name , "_obs" 
            
            if (oldg_median):
                cleanCLs(oldg_median)
                bands.WriteTObject(oldg_median.Clone(name+"_"+oldname+"_median"), name+"_"+oldname+"_median")
                print "Imported " , oldname , " result " , name << "_median"
            
            if (oldg_medi95):
                cleanCLs(oldg_medi95)
                bands.WriteTObject(oldg_medi95.Clone(name+"_"+oldname+"_median_95"), name+"_"+oldname+"_median_95")
                print "Imported " , oldname , " result " , name , "_median_95"
            
        
    
    oldBands.Close()


In [8]:
def importLandS(bands):
    importLandS(bands, "mlz_comb_lands",    "lands/comb_maxllfit.root", 1, 0)
    importLandS(bands, "mlz_combe_lands",   "lands/comb_maxllfit_ebereso.root", 1, 0)
    importLandS(bands, "pvala_comb_lands",  "lands/comb_pvalue.root",   1, 0)
    importLandS(bands, "pvala_comb_lands",  "lands/comb_pvalueSB.root", 0, 1)
    importLandS(bands, "pvala_combe_lands", "lands/comb_pvalue_ebereso.root", 1, 0)

In [9]:
def makebands( which=0):
    global prefix
    #gROOT.LoadMacro("$CMSSW_BASE/src/HiggsAnalysis/CombinedLimit/test/plotting/bandUtils.cxx+")

    if which == 1: prefix = "SM4_"
    elif which == 2: prefix = "FF_"
    

    bands = ROOT.TFile(("results/%sbands.root" % prefix),"RECREATE")

    print "Will save bands to " , bands.GetName()
    #from bandUtils import ObsAvgMode
    #ObsAvgMode = Enum(['MeanObs', 'LogMeanObs', 'MedianObs'])
    obs_avg_mode = ObsAvgMode.MedianObs
    do_bands_nosyst = False
    do_bands_mean   = False
    do_bands_ntoys  = False
    do_bands_asimov = False
    halfint_masses = True

    nwhat = 13
    what = [ "pla", "pvala", "pvala", "pval", "acls" , "ml", "mlz", "cls",  "bayes", "smcls", "smacls", "acls90", "acls99"]
    WHAT = [ "PLC", "PLP",   "PLPE",  "PVAL", "ASCLS", "ML", "MLZ", "FREQ", "BAYES", "SMCLS", "SMASCLS", "ASCLS90", "ASCLS99"]

    
    nwho = 20
    who = [ "comb", "htt", "vhtt", "httm", "htt0", "hzz", "hzz2l2t", "hzz2l2nu", "hzz2l2q_all", "hzz4l", "hww", "vhww3l", "hww2l", "hgg_novbf", "hgg_vbf", "hgg", "vhbb", "combs", "combp_low", "combl_low" ]
    WHO = [ "COMB", "HTT", "VHTT", "HTTM", "HTT0", "HZZ", "HZZ2L2T", "HZZ2L2NU", "HZZ2L2Q",     "HZZ4L", "HWW", "VHWW3L", "HWW2L", "HGG_NOVBF", "HGG_VBF", "HGG", "VHBB", "COMBS", "COMBP", "COMBL" ]

    for i in xrange(nwho) : # for (int i = 0; i < nwho; ++i):
        #//if (!TString(who[i]).Contains("comb")) continue
        if (not "hzz4l" in who[i]) : continue
        #//doFc(bands, TString::Format("fc_%s",who[i]), TString::Format("higgsCombine%s_FC.root", WHO[i]))
        #//continue
        for j in xrange(nwhat) : # for (int j = 0; j < nwhat; ++j):
            if ( what[j] != "acls" and what[j] != "pvala" ) : continue
            print "doIt what = " , what[j] , " who = " , who[i] 
            doIt(bands, ("%s_%s" % (what[j], who[i])),
                        ("results/higgsCombine%s%s_%s.root" % (prefix, WHO[i], WHAT[j])))
        
        #//break
        #if 0
        #if (not ROOT.gSystem.AccessPathName(("timecls_%s.txt" %who[i]))):
        #    time = ROOT.TGraphErrors(("timecls_%s.txt" % who[i]), "Mass: %lg Time: %lg +/- %lg s")
        #    time.SetName(("timecls_%s_obs" % who[i]))
        #    bands.WriteTObject(time)
        #
        #endif
    

    #if 1 == 0 : ##if 0
    #const int npvflav = 9
    #pvflav[npvflav] = { "M2M", "S", "SZ3", "BZ1", "BZ", "TQ0", "TQ03", "TQ05N", "TQ05" 
    #for (int i = 0; i < npvflav; ++i):
    #    doIt(bands, TString::Format("pvala_%s_comb", pvflav[i]),
    #                TString::Format("results/higgsCombine%sCOMB_PLP.%s.root", prefix.Data(), pvflav[i]))
    
    ##endif
    ##if 0
    #if (which == 0):
    #    importLine(bands, "tevatron_obs",    "tevatron.obs.txt");    
    #    importLine(bands, "tevatron_median", "tevatron.exp.txt");    
    #    std::cout  << "Imported Tevatron results" << std::endl
    #    importCloned(bands, "results/bands.paper.root", "paper", nwhat, what, nwho, who)
    
    ##endif

    ##if 0
    #for (int j = 0; j < nwhat; ++j):
    #    w = what[j]
    #    cutBands(bands, w+"_hzz2l2q_all", w+"_hzz2l2q_low", 130, 164)
    #    cutBands(bands, w+"_hzz2l2q_all", w+"_hzz2l2q",     200, 600)
    #    cutBands(bands, w+"_hzz", w+"_combp_high", 150.5, 600)
    #    cutBands(bands, w+"_hww", w+"_combl_high", 145.5, 600)
    #    pasteBands(bands, w+"_combl_low", w+"_combl_high", w+"_combl")
    #    pasteBands(bands, w+"_combp_low", w+"_combp_high", w+"_combp")
    
    ##endif
    if (which == 0):
        cutBands(bands, "smcls_comb",       "smcls_comb_new", 110, 200)
        cutBands(bands, "smcls_comb_paper", "smcls_comb_old", 200, 600)
        pasteBands(bands, "smcls_comb_new", "smcls_comb_old", "smcls_comb_patch")
        cutBands(bands, "cls_comb",       "cls_comb_new", 110, 200)
        cutBands(bands, "cls_comb_paper", "cls_comb_old", 200, 600)
        pasteBands(bands, "cls_comb_new", "cls_comb_old", "cls_comb_patch")


In [10]:
#root -b -l -q makeBands.cxx
makebands()

Will save bands to  results/bands.root
doIt what =  pvala  who =  hzz4l
 name  pvala_hzz4l  access rootfile  results/higgsCombineHZZ4L_PLP.root  result =  False
DEBUG checking  results/higgsCombineHZZ4L_PLP.root
TFile**		results/higgsCombineHZZ4L_PLP.root	
 TFile*		results/higgsCombineHZZ4L_PLP.root	
  KEY: TDirectoryFile	toys;1	toys
  KEY: TTree	limit;1	limit
DEBUG end of checking  results/higgsCombineHZZ4L_PLP.root
Band  pvala_hzz4l_nosyst_mean  missing 1
Band  pvala_hzz4l_nosyst_median  missing 1
Band  pvala_hzz4l_nosyst_obs  missing 2
Band  pvala_hzz4l_nosyst_ntoys  missing 2
Band  pvala_hzz4l_nosyst_asimov  missing 2
Band  pvala_hzz4l_mean  missing 1
Band  pvala_hzz4l_median  missing 1
Band  pvala_hzz4l_obs  missing 2
Band  pvala_hzz4l_ntoys  missing 2
Band  pvala_hzz4l_asimov  missing 2
Trying to do  pvala_hzz4l
MISSING  pvala_hzz4l _mean and  pvala_hzz4l _obs
doIt what =  pvala  who =  hzz4l
 name  pvala_hzz4l  access rootfile  results/higgsCombineHZZ4L_PLPE.root  result =  Fals

In [48]:
dataset={}
#dataset.append([])
for mass in xrange(10):
    #try:
    #    dim = len(dataset[i])
    #except:
    #    dataset.append([])
    if mass in dataset:
        dataset[mass].append(i+1)
    else:
        dataset[mass] = []
        dataset[mass].append(i+1)
        #[i].append(i+1)    
print len(dataset), len(dataset[0]) 
print dataset

10 1
{0: [10], 1: [10], 2: [10], 3: [10], 4: [10], 5: [10], 6: [10], 7: [10], 8: [10], 9: [10]}


In [28]:
arr = []
arr.append([])
arr[0].append('aa1')
arr[0].append('aa2')

print arr

[['aa1', 'aa2']]


In [26]:
mydict[1].append(2)
print mydict

AttributeError: 'int' object has no attribute 'append'

In [52]:
from bandUtils import *
# TEST
band_safety_crop = 0
use_precomputed_quantiles = False
precomputed_median_only = False
zero_is_valid = False
seed_is_channel = False
halfint_masses  = False #// find the halfling!                                  
doSyst = 1
#doSyst = 0
whichChannel = 0
type = BandType.Median 
root_file="results/higgsCombineHZZ4L_PLP.root"
root_file="results/higgsCombineHZZ4L_ASCLS.root"
root_file="results/higgsCombineHZZ4L_PLPE.root"
file = ROOT.TFile(root_file)
t = file.Get("limit")
if not t : t = file.Get("test") #// backwards compatibility
if not t: 
    print "TFile " , file.GetName() , " does not contain the tree" 

dataset = {} # std::map<int,std::vector<double> >  dataset
errors = {} # std::map<int,std::vector<double> >  errors
#obsValues # std::map<int,double>                obsValues
obsValues = {}



n = t.GetEntries()
#print "DEBUG printing tree "
print " dataset ", len(dataset)
#t.Print()
for i in xrange(n): # for (size_t i = 0, n = t.GetEntries(); i < n; ++i):
        t.GetEntry(i)
        mass = t.mh ; limit = t.limit
        if t.GetBranch("limitErr") : limitErr = t.limitErr
        if (t.GetBranch("t_cpu") != 0):
            t_cpu = t.t_cpu
            t_real = t.t_real
        if (use_precomputed_quantiles):
            if (t.GetBranch("quantileExpected") == 0): 
                print "TFile " , file.GetName() , " does not have precomputed quantiles" 
                #return 0
            quant = t.quantileExpected
        syst = t.syst
        if seed_is_channel : iChannel = t.iSeed
        else : iChannel = t.iChannel
        iToy = t.iToy
        iMass = int(mass*100)
        #try:
        #    dim = len(dataset[iMass])
        #except:
        #    dataset.append([])
        #dataset[iMass].append(limit) # .push_back(limit)
        #dataset.append([iMass,limit])
        #dataset.append([iMass,9999.9])
        if iMass in dataset:
            dataset[iMass].append(limit)
            dataset[iMass].append(9999.9)
        else:
            dataset[iMass] = []
            dataset[iMass].append(limit)
            dataset[iMass].append(99999.9)
        #errors[iMass].append(limitErr) # .push_back(limitErr)
        #print i, iMass, " limit ",limit, syst, iChannel
        #printf("%6d mh=%.1f %6d limit=%8.3f +/- %8.3f toy=%5d quant=% .3f\n" %( i, mass, imass, limit, limitErr, iToy, quant))
        if (syst != doSyst) :  print "1 ";         continue
        if (iChannel != whichChannel) : print "2 "; continue
        if      (type == BandType.Asimov) :
            if (iToy != -1) : print "3 ";  continue 
        elif (type == BandType.Observed): 
            if (iToy !=  0) : print "4 ";  continue
        elif (type == BandType.ObsQuantile and iToy == 0): 
            obsValues[iMass] = limit
            print "5 "; continue 
        elif (iToy <= 0 and not use_precomputed_quantiles) : continue ; print "6 ", iToy; continue
        if (limit == 0 and not zero_is_valid) : print "7 "; continue
        if (type == BandType.MeanCPUTime): 
            if (limit < 0) : continue
            limit = t_cpu # limit = t_cpu
        
        if (type == BandType.MeanRealTime): 
            if (limit < 0) : continue
            limit = t_real
        
        if (use_precomputed_quantiles):
            if (type == BandType.CountToys)   : break # return 0
            if (type == BandType.Mean)        : break # return 0
            #//std::cout << "Quantiles. What should I do " << (type == Observed ? " obs" : " exp") << std::endl
            if (type == BandType.Observed and quant > 0) :continue
            if (type == BandType.Median):
                if (fabs(quant - 0.5) > 0.005 and fabs(quant - (1-width)/2) > 0.005 and fabs(quant - (1+width)/2) > 0.005):
                    #//std::cout << " don't care about " << quant << std::endl
                    continue
                 #else
                 #   #//std::cout << " will use " << quant << std::endl
                
            
        print iMass, " limit ",limit
        
        #dataset[iMass].append(limit) # .push_back(limit)
        #errors[iMass].append(limitErr) # .push_back(limitErr)
#    #iMass = int(t.mass*100)
#    #printf("%6d mh=%.1f  limit=%8.3f +/- %8.3f toy=%5d quant=% .3f\n" % (i, mass, limit, limitErr, iToy, quant))
#print " dataset at the end ",len(dataset)
print dataset
if 1 == 1:
    for idx, data in enumerate(dataset) :
        #data is it.second # std::vector<double> &data = it.second
        #        data=data0[1:]
        nd=len(data) # int nd = data.size()
        print " nd ", nd, "idx ", idx, " data ", data

TypeError: object of type 'int' has no len()

 dataset  0
{11200: [0.1510777815424868, 99999.9], 17600: [0.0007077066013611862, 99999.9], 20400: [1.0870535633821438e-10, 99999.9], 18400: [2.6471251350562748e-11, 99999.9], 19600: [7.272770693623982e-13, 99999.9], 18800: [1.6737329165380259e-15, 99999.9], 11400: [0.09443718302397747, 99999.9], 12300: [7.158047395093458e-08, 99999.9], 18000: [4.209169951151133e-06, 99999.9], 11500: [0.0686976802816395, 99999.9], 18200: [2.3213982608764528e-08, 99999.9], 12400: [1.1755053614420032e-08, 99999.9], 14200: [1.6991338935573894e-17, 99999.9], 21000: [4.297625528732554e-10, 99999.9], 19400: [2.336055925261373e-12, 99999.9], 12600: [8.933820789647979e-11, 99999.9], 18600: [1.9556608376997088e-13, 99999.9], 12700: [1.4024772254899257e-11, 99999.9]}


In [18]:
print dataset

[]


In [53]:
def theBand(file, doSyst, whichChannel, type, width=0.68):
    if not file : return 0
    t = file.Get("limit")
    if not t : t = file.Get("test") #// backwards compatibility
    if not t: 
        print "TFile " , file.GetName() , " does not contain the tree" 
        return 0 

    dataset = {} # std::map<int,std::vector<double> >  dataset
    errors = {} # std::map<int,std::vector<double> >  errors    
    obsValues = {} # std::map<int,double>                obsValues
    
    n = t.GetEntries()
    #print "DEBUG printing tree "
    #t.Print()
    for i in xrange(n): # for (size_t i = 0, n = t.GetEntries(); i < n; ++i):
        t.GetEntry(i)

        mass = t.mh ; limit = t.limit
        if t.GetBranch("limitErr") : limitErr = t.limitErr
        if (t.GetBranch("t_cpu") != 0):
            t_cpu = t.t_cpu
            t_real = t.t_real
        if (use_precomputed_quantiles):
            if (t.GetBranch("quantileExpected") == 0): 
                print "TFile " , file.GetName() , " does not have precomputed quantiles" 
                #return 0
            quant = t.quantileExpected
        syst = t.syst
        if seed_is_channel : iChannel = t.iSeed
        else : iChannel = t.iChannel
        iToy = t.iToy
        
        iMass = int(mass*100)
        #//printf("%6d mh=%.1f  limit=%8.3f +/- %8.3f toy=%5d quant=% .3f\n", i, mass, limit, limitErr, iToy, quant)
        if (syst != doSyst) :           continue
        if (iChannel != whichChannel) : continue
        if      (type == BandType.Asimov) :
            if (iToy != -1) : continue 
        elif (type == BandType.Observed): 
            if (iToy !=  0) : continue
        elif (type == BandType.ObsQuantile and iToy == 0): 
            #obsValues[iMass] = limit
            if iMass in obsValues:
                obsValues[iMass] = limit
            else:
                obsValues[iMass] = []
                obsValues[iMass] = limit
            continue 
        elif (iToy <= 0 and not use_precomputed_quantiles) : continue
        if (limit == 0 and not zero_is_valid) : continue
        if (type == BandType.MeanCPUTime): 
            if (limit < 0) : continue
            limit = t_cpu # limit = t_cpu
        
        if (type == BandType.MeanRealTime): 
            if (limit < 0) : continue
            limit = t_real
        
        if (use_precomputed_quantiles):
            if (type == BandType.CountToys)   :return 0
            if (type == BandType.Mean)        :return 0
            #//std::cout << "Quantiles. What should I do " << (type == Observed ? " obs" : " exp") << std::endl
            if (type == BandType.Observed and quant > 0) :continue
            if (type == BandType.Median):
                if (fabs(quant - 0.5) > 0.005 and fabs(quant - (1-width)/2) > 0.005 and fabs(quant - (1+width)/2) > 0.005):
                    #//std::cout << " don't care about " << quant << std::endl
                    continue
                 #else
                 #   #//std::cout << " will use " << quant << std::endl
                
            
        
        dataset[iMass].append(limit) # .push_back(limit)
        errors[iMass].append(limitErr) # .push_back(limitErr)
        if iMass in dataset:
            dataset[iMass].append(limit)
            errors[iMass].append(limitErr)
        else:
            dataset[iMass] = []
            dataset[iMass].append(limit)
            errors[iMass] = []
            errors[iMass].append(limitErr)
    #print "DEBUG dataset ",len(dataset), dataset
    tge = ROOT.TGraphAsymmErrors () # new () # TGraphAsymmErrors
    ip = 0
    #for value in values:
    #   # Print each row's length and its elements.
    #   print(len(value), value)
    # for (std::map<int,std::vector<double> >::iterator it = dataset.begin(), ed = dataset.end(); it != ed; ++it):
    #for idx, data in enumerate(dataset) :
    for idx, data in dataset.iteritems():
        #data is it.second # std::vector<double> &data = it.second
        nd=len(data) # int nd = data.size()
        data.sort() # std::sort(data.begin(), data.end())
        if nd % 2 == 0 :
            median = 0.5*(data[nd/2]+data[nd/2+1])
        else :
            median = data[nd/2]
            
        if (band_safety_crop > 0):
            data2 = [] # std::vector<double> data2
            for j in xrange(nd): # for (int j = 0; j < nd; ++j):
                if (data[j] > median*band_safety_crop and data[j] < median/band_safety_crop):
                    data2.append(data[j]) # data2.push_back(data[j])
                
            
            data2.swap(data)
            datatmp=data2, data
            data=datatmp[0] 
            data2=datatmp[1]
            nd = len(data) #data.size()
            if nd % 2 == 0 : median = 0.5*(data[nd/2]+data[nd/2+1])
            else : median =  data[nd/2]
        mean = 0
        for j in xrange(nd):
            mean += data[j]
        mean /= nd
        summer68 = data[floor(nd * 0.5*(1-width)+0.5)]
        winter68 =  data[min(int(floor(nd * 0.5*(1+width)+0.5)), nd-1)]
        if (use_precomputed_quantiles and type == BandType.Median):
            if (precomputed_median_only and len(data) == 1):
                mean = median = summer68 = winter68 = data[0]
            elif (len(data) != 3): 
                print "Error for expected quantile for mass " , idx , ": size of data is " , len(data)
                continue
            else :
                mean = median = data[1]; summer68 = data[0]; winter68 = data[2]
            
        
        x = mean
        #switch (type):
        if type == BandType.MeanCPUTime: pass
        elif type == BandType.MeanRealTime: pass
        elif type == BandType.Mean: x = mean; break
        elif type == BandType.Median: x = median; break
        elif type == BandType.CountToys: x = summer68 = winter68 = nd; break
        elif type == BandType.Asimov: #// mean (in case we did it more than once), with no band
            if obs_avg_mode == mean : winter68 = mean
            else : winter68 = median
            x = summer68 = winter68 # = (obs_avg_mode == mean ? mean : median)
            break
        elif type == BandType.Observed:
            x = mean
            if (nd == 1):
                if (len(errors[idx]) == 1):
                    summer68 = mean - errors[idx][0]
                    winter68 = mean + errors[idx][0]
                else :
                    #// could happen if limitErr is not available
                    summer68 = winter68 = mean

            else : #// if we have multiple, average and report rms (useful e.g. for MCMC)
                #switch (obs_avg_mode):
                if obs_avg_mode == ObsAvgMode.MeanObs:   x = mean; break
                elif obs_avg_mode ==  ObsAvgMode.MedianObs: x = median; break
                elif obs_avg_mode ==  ObsAvgMode.LogMeanObs:
                    x = 0
                    for j in xrange(nd) : x += log(data[j]) #for (int j = 0; j < nd; ++j): x += log(data[j]); 
                    x = exp(x/nd)
                    break

                rms = 0
                for j in xrange(nd) : rms += (x-data[j])*(x-data[j]) # for (int j = 0; j < nd; ++j): rms += (x-data[j])*(x-data[j]); 
                rms = sqrt(rms/(nd*(nd-1)))
                summer68 = mean - rms
                winter68 = mean + rms

            break
        elif type == BandType.AdHoc:
            x = summer68 = winter68 = mean
            break
        elif type == BandType.Quantile: #// get the quantile equal to width, and it's uncertainty
            x = data[floor(nd*width+0.5)]
            summer68 = x - quantErr(nd, data, width)
            winter68 = x + (x-summer68)
            break
        elif type == BandType.ObsQuantile:   
            #if ( idx == ( len(obsValues) - 1)) : continue # ????? #if (obsValues.find(it.first) == obsValues.end()) continue
            # we need to check if idx exist in obsValues
            if not idx in obsValues : continue
            passed = 0
            failed = 0
            for i in xrange(nd): # for (int i = 0; i < nd and data[i] <= obsValues[it.first]; ++i):
                if data[i] > obsValues[idx] : continue
                failed += 1

            passed = nd - failed; x = double(passed)/nd
            alpha = (1.0 - .68540158589942957)/2
            if passed == 0 : summer68 = 0.0
            else : summer68 = ROOT.Math.beta_quantile(   alpha, passed,   failed+1 )
            if failed == 0 : winter68 = 1.0
            else : winter68 = ROOT.Math.beta_quantile( 1-alpha, passed+1, failed   )
            break

        #// end switch
        tge.Set(ip+1)
        tge.SetPoint(ip, it.first*0.01, x)
        tge.SetPointError(ip, 0, 0, x-summer68, winter68-x)
        ip += 1
    
    return tge


In [ ]:
#root_file="results/higgsCombineHZZ4L_PLP.root"
#file = ROOT.TFile(root_file)
filename="results/higgsCombineHZZ4L_PLP.root"
file = ROOT.TFile.Open(filename)
band68 = theBand(file, 0, 0, 0, 0.68)

In [ ]:
#THE END

In [3]:
#from bandUtils import *
which = 0
#def plots(which):
if 1 == 1:
    print 1
    #gROOT.LoadMacro("$CMSSW_BASE/src/HiggsAnalysis/CombinedLimit/test/plotting/bandUtils.cxx+")
    #ROOT.gSystem.Load("/cvmfs/sft.cern.ch/lcg/releases/LCG_85swan3/vdt/0.3.6/x86_64-slc6-gcc49-opt/lib/libvdt.so")
    #os.environ['LD_LIBRARY_PATH']='/cvmfs/cms.cern.ch/slc6_amd64_gcc493/external/gcc/4.9.3/lib64:/cvmfs/sft.cern.ch/lcg/views/LCG_85swan3/x86_64-slc6-gcc49-opt/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_85swan3/x86_64-slc6-gcc49-opt/lib64:$LD_LIBRARY_PATH'
    #os.environ['LD_LIBRARY_PATH']='/cvmfs/cms.cern.ch/slc6_amd64_gcc491/external/gcc/4.9.1-cms/lib64:/cvmfs/sft.cern.ch/lcg/views/LCG_85swan3/x86_64-slc6-gcc49-opt/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_85swan3/x86_64-slc6-gcc49-opt/lib64:$LD_LIBRARY_PATH'
    #ROOT.gSystem.Load("/cvmfs/cms.cern.ch/slc6_amd64_gcc493/external/gcc/4.9.3/lib64/libstdc++.so")
    #ROOT.gROOT.LoadMacro("../HiggsAnalysis/CombinedLimit/test/plotting/bandUtils.cxx+")
    theBand()
    print 2
    inputPrefix = ""
    #switch (which)  {
    if which == 1:
        inputPrefix = "SM4_" ; pdefs.SM="SM4"; 
        pdefs.LEESPAM_2L = "#splitline{Global p-value ~0.5}{for 110-600 GeV range}"
        #        break
    elif which == 2:
        pdefs.inputPrefix = "FF_"  ; pdefs.SM="FP";  
        pdefs.LEESPAM_2L = "#splitline{Global significance 1.1#sigma}{for 110-300 GeV range}"
        #        break
    #}
    print 4

    loadMasses("masses.txt")
    print 5
    globalPrefix0 = inputPrefix+"plots/"
    pdefs.globalPrefix =  globalPrefix0+"/"
    ROOT.gStyle.SetOptTitle(0)
    print "cp results/"+inputPrefix+"bands.root "+inputPrefix+"bands.work.root"
    ROOT.gSystem.Exec("/bin/cp results/"+inputPrefix+"bands.root "+inputPrefix+"bands.work.root")
    print 6
  
    #//TFile::Open(inputPrefix+"bands.work.root", "UPDATE")
    workband = ROOT.TFile.Open(inputPrefix+"bands.work.root", "UPDATE")
    print 7

    print inputPrefix
    #// WRITE OUT THE GRIDS TO RUN CLS AND BAYESIAN
    #////writeGrids(inputPrefix) return; 
    writeGrid("hzz4l","")
    #workband.Close()

    #workband.ls()

    #if not workband.Get("pval_comb"):
    #    print "NOTOK"
    #else:
    #    print "OK"
    #if workband.Get("pval_comb") is None:
    #    print "nill"
    #else:
    #    print "OK"
    ##workband.Get("pvala_coms")
    

1
2
4
5
cp results/bands.root bands.work.root
6
7

Asked for grid  acls_hzz4l


In [4]:
    #from bandUtils import *
    #////return 
    #switch (which):
    #print 7.1
    if which == 0:
        print "gFile = " , ROOT.gFile.GetName() #, " gFile.GetN() ", ROOT.gFile.GetN()
        #//*test = (*) gFile.Get("acls_hzz4l_obs")
        #//bandIn="pval_comb"
        #//bandOut="pval_comb125"
        #//selectedPointsBands("pval_comb")
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb125", 125.0)
        #selectedPointsBands(workband, "pval_comb", "pval_comb125", 125.0)
        #//selectedPointsBands(gFile, "pval_comb", "pval_comb125", 125.0)
        #//selectedPointsBands(gFile, "pval_comb", "pval_comb12x", 125.0, 124.5, 124.0, 125.5, 126.0)
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb12x", 125.0, 124.5, 124.0, 125.5, 126.0)
        #selectedPointsBands(workband, "pval_comb", "pval_comb12x", 125.0, 124.5, 124.0, 125.5, 126.0)
        #//selectedPointsBands(gFile, "pval_comb", "pval_comb12x", 125.0)
        #break
    elif which == 1:
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb125", 119.0)
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb12x", 119.0, 120.0, 118.0, 121.0, 117.0)
        #break
    elif which == 2:
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb125", 126.0)
        selectedPointsBands(ROOT.gFile, "pval_comb", "pval_comb12x", 126.0, 124.0, 125.0, 127.0, 128.0)
        #break
    
    print 8


gFile =  bands.work.root
selectedPointsBands 
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
selectedPointsBands 
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
band is nil
8


In [5]:
    #pdefs.c1 = ROOT.TCanvas("c1","c1")
    rectangleCanvas()


In [6]:
    print 9

    #//#if 0
    #// PAPER PLOTS, keep it simple
    #squareCanvas(/*grid x,y=*/0,0); x_zoom = 145;
    squareCanvas(0,0) ; pdefs.x_zoom = 145;
    pdefs.globalPrefix = globalPrefix0+"/pre-paper/sqr_"
    pdefs.noLineStyles = True #// always use solid lines

    #// fig 1
    #//drawSMCLs(TString::Format("smcls_%s", "comb"))

    #// fig 2
    #//drawOneCLs(TString::Format("cls_%s",  "comb"))

    #// fig 3
    #//bockjoo drawCombObs("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann, nchann)
    pdefs.lineAt1Style = 2
    print "chann ", pdefs.chann, " nchann ",pdefs.nchann
    #drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann, nchann, /*obs=*/False, /*combined=*/False)
    drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+pdefs.SM+"}", pdefs.chann, pdefs.nchann, False, False)
    pdefs.lineAt1Style = 1



9
chann  ['comb', 'vhbb', 'htt', 'hgg', 'hww', 'hzz', 'combp', 'combs', 'combl', 'httm', 'vhtt', 'vhww3l', 'htt0', 'hww2l', 'hzz4l', 'hzz2l2t', 'hzz2l2q_low', 'hzz2l2q', 'hzz2l2nu']  nchann  14
acls_comb_obs not found in  bands.work.root


In [7]:
    #// fig 4
    pdefs.forceYmin = 0.08 ; pdefs.forceYmax = 12;
    pdefs.channelSpamOnRightHandSide = True
    drawOneCLs("acls_%s" %  "hzz4l")
    #//drawOneCLs(TString::Format("acls_%s",  "combl_full"))
    pdefs.channelSpamOnRightHandSide = False
    pdefs.forceYmin = 0 ; pdefs.forceYmax = 0;


Asked for grid  acls_hzz4l
missingPoints m68
justSave saved :  plots//pre-paper/sqr_acls_hzz4l.eps
justSave saved :  plots//pre-paper/sqr_acls_hzz4l.png
justSave saved :  plots//pre-paper/sqr_acls_hzz4l.pdf
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_logx.eps
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_logx.png
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_logx.pdf
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_zoom.eps
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_zoom.png
justSave saved :  plots//pre-paper/sqr_acls_hzz4l_zoom.pdf


Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_acls_hzz4l.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_acls_hzz4l.pdf has been created
Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_acls_hzz4l_logx.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_acls_hzz4l_logx.pdf has been created
Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_acls_hzz4l_zoom.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_acls_hzz4l_zoom.pdf has been created


In [13]:
    print "drawCombPVal chann " , pdefs.chann , " nchann " , pdefs.nchann
    #//fig 5a
    drawCombPVal("pvala", pdefs.chann, pdefs.nchann)

drawCombPVal chann  ['comb', 'vhbb', 'htt', 'hgg', 'hww', 'hzz', 'combp', 'combs', 'combl', 'httm', 'vhtt', 'vhww3l', 'htt0', 'hww2l', 'hzz4l', 'hzz2l2t', 'hzz2l2q_low', 'hzz2l2q', 'hzz2l2nu']  nchann  14
pvala_comb_obs is null in  bands.work.root


In [14]:
    #// bockjoo
    print "drawOnePlot " , ("pvala_%s" % "hzz4l")
    drawOnePlot("pvala_%s" % "hzz4l")


drawOnePlot  pvala_hzz4l
 myspam =  CMS Preliminary
#sqrt{s} = 7 TeV
H #rightarrow ZZ #rightarrow 4l, L = 4.7 fb^{-1}
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l.eps
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l.png
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l.pdf
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_logx.eps
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_logx.png
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_logx.pdf
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_zoom.eps
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_zoom.png
justSave saved :  plots//pre-paper/sqr_pvala_hzz4l_zoom.pdf


Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_pvala_hzz4l.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_pvala_hzz4l.pdf has been created
Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_pvala_hzz4l_logx.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_pvala_hzz4l_logx.pdf has been created
Info in <TCanvas::Print>: eps file plots//pre-paper/sqr_pvala_hzz4l_zoom.eps has been created
Info in <TCanvas::Print>: pdf file plots//pre-paper/sqr_pvala_hzz4l_zoom.pdf has been created


In [1]:

    print "drawOnePlot " , "mlz_hzz4l"
    #//fig 5b
    drawOnePlot(TString::Format("mlz_%s", "hzz4l")) 

    #// fig 6
    globalPrefix = globalPrefix0+"/pre-paper/"
    cccPlot("mlz", 119.5,  chann, nchann, "", -1.0, 4.0)
    cccPlot("mlz", 124.0,  chann, nchann, "", -1.0, 3.0)
    return
    #//#endif

#if 0
    #// PAS PLOTS, keep it simple
    squareCanvas(/*grid x,y=*/0,0) 
    globalPrefix = globalPrefix0+"/sqr_"
    noLineStyles = True #// always use solid lines
    lessSpacesInLegends = True #// compact legends (we only use decay modes combined)

    if (which == 0):
        drawSMCLs(TString::Format("smcls_%s", "comb"))

        drawOneCLs(TString::Format("cls_%s",  "comb"))

        drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, True,  True, "_zzc")
        
        drawCombPVal("pvala", chann2, nchann2, "_zzc")
        drawOnePlot(TString::Format("mlz_%s", "comb")) 
        
        globalPrefix = globalPrefix0+"/"
        cccPlot("mlz", 119.5,  chann, nchann, "", -2.0, 4.0)
        cccPlot("mlz", 125.0,  chann, nchann, "", -1.0, 4.0)
    } else if (which == 2):
        drawOneCLs(TString::Format("acls_%s",  "comb"))

        drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, True,  True, "_zzc")
        
        drawCombPVal("pvala", chann2, nchann2, "_zzc")
        drawOnePlot(TString::Format("mlz_%s", "comb")) 
    }
    return
#endif


#if 0
    squareCanvas() globalPrefix =  globalPrefix0+"/"; c1.SetGridy(0); 
    switch (which):
        case 0:
        for (int gridx = 0 gridx < 2; ++gridx):
            globalPrefix =  globalPrefix0+(gridx ? "/grid_" : "/")
            c1.SetGridx(gridx)
            cccPlot("mlz", 119.5,  chann2, nchann2, "", -3.0, 4.0)
            cccPlot("mlz", 124.0,  chann2, nchann2, "", -1.0, 4.0)
            cccPlot("mlz", 125.0,  chann2, nchann2, "", -1.0, 4.0)
            cccPlot("mlz", 136.,  chann2, nchann2, "", -1.0, 7.0)
            cccPlot("mlz", 144.,  chann2, nchann2, "", -1.5, 5.0)
        }
        break
        case 2:
        for (int gridx = 0 gridx < 2; ++gridx):
            globalPrefix =  globalPrefix0+(gridx ? "/grid_" : "/")
            c1.SetGridx(gridx)
            cccPlot("mlz", 119.5,  chann2, nchann2, "", -1.0, 5.0)
            cccPlot("mlz", 124.0,  chann2, nchann2, "", -1.0, 4.0)
            cccPlot("mlz", 125.0,  chann2, nchann2, "", -1.0, 4.0)
            cccPlot("mlz", 126.0,  chann2, nchann2, "", -1.0, 4.0)
            cccPlot("mlz", 130.5,  chann2, nchann2, "", -1.0, 7.0)
        }
        break
    }
    return
#endif

    *prefixes[4] = { "", "sqr_", "sqr_grid_", "grid_" }
    const int   grids   [4] = {  0, 0,      1,           1,      }
    const int   squared [4] = {  0, 1,      1,           0,      }
    for (int version = 0 version < 4; ++version):
        #//if (version != 1) continue
        if (squared[version]) squareCanvas(grids[version],grids[version])
        else rectangleCanvas(grids[version],grids[version])

        globalPrefix =  globalPrefix0+"/"+prefixes[version]
#if 1   
        noLineStyles = True
        #//if (gFile.Get("acls_comb_obs")) drawCombObs("acls",  "95% CL limit on #sigma/#sigma_{"+SM+"}", chann4, nchann4, True)
        #//if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann4, nchann4)
        #//if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann4, nchann4, False)
        noLineStyles = False
	std::cout << 10 << std::endl

        #// ----- ZZ combined ------
        lessSpacesInLegends = True 
        noLineStyles = True
        if (gFile.Get("acls_comb_obs")) drawCombObs("acls",  "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, True,        "_zzc") 
        if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, True,  True, "_zzc")
        if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, False, True, "_zzc")
	std::cout << 11 << std::endl
	noLineStyles = True
        #//if (gFile.Get("acls_comb_obs")) drawCombObs("acls",  "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, True,        "_zzc_solid") 
        #//if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann2, nchann2, False, True, "_zzc_solid")
        noLineStyles = False
        lessSpacesInLegends = False
	std::cout << 12 << std::endl

        #// ----- Two SubCombs -----
        #//if (gFile.Get("acls_comb_obs")) drawCombBoth("acls", "95% CL limit on #sigma/#sigma_{"+SM+"}", chann3, nchann3, True, True, "_c3")
        #// ====== PVALS ======
#endif
        noLineStyles = True
        if (gFile.Get("pvala_comb_obs")) drawCombPVal("pvala", chann, nchann)
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc"); lessSpacesInLegends = False; }
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc_t",  "pval_comb125"); lessSpacesInLegends = False; }
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc_t2", "pval_comb12x"); lessSpacesInLegends = False; }
        doLEESPAM = True
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc_gz"); lessSpacesInLegends = False; }
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc_t_gz", "pval_comb125"); lessSpacesInLegends = False; }
        if (gFile.Get("pvala_comb_obs")): lessSpacesInLegends = True drawCombPVal("pvala", chann2, nchann2, "_zzc_t2_gz", "pval_comb12x"); lessSpacesInLegends = False; }
        doLEESPAM = False
        #//if (gFile.Get("pvala_comb_obs")) drawCombPVal("pvala", chann3, nchann3, "_c3")
        noLineStyles = False
        #//drawCombMuHat("mlz", 110, 150, -2.5, 5, "")  
        #//drawCombMuHat("mlz", 110, 150, -2.5, 5, "_nofill", 0)  
        #//drawCombMuHat("mlz", 114, 132, -2, 4, "_zoom")  
        #//drawCombMuHat("mlz", 114, 132, -2, 4, "_zoom_nofill", 0)  


        for (int i = 0 i < nchann; ++i):
            if (!TString(chann[i]).Contains("comb")): 
                globalPrefix =  globalPrefix0+"/subchannels/"+prefixes[version]
            } else { 
                globalPrefix =  globalPrefix0+"/"+prefixes[version]
            }

            #//if (!TString(chann[i]).Contains("comb")) continue
            #//globalPrefix =  globalPrefix0+"/private/"

            #//if (which <= 1 and (TString(chann[i]) != "comb" and TString(chann[i]) != "hgg")) continue
            #//if (i < 1+5+3+3) continue
            if (TString(chann[i]) != "comb") continue
            #//if (!TString(chann[i]).Contains("comb")) continue
            #//if (TString(chann[i]) != "vhtt") continue
#if 0       #//---- Asymptotics ----
            #//CLs_debug_apriori_grid = TString(chann[i]).Contains("comb")
	    std::cout << 13 << std::endl
	    std::cout << chann[i] << std::endl
            drawOneCLs(TString::Format("acls90_%s", "hzz4l"))
            drawOneCLs(TString::Format("acls99_%s", "hzz4l"))
	    
            #//drawOnePVal(TString::Format("pvala_%s", chann[i]), "Local p-value", /*include_toys=*/False)
	    std::cout << 14 << std::endl
	    #//drawOnePlot(TString::Format("mlz_%s", chann[i])) 
            #////drawOnePValFitCollage(TString::Format("%s", chann[i]), "mlz")
#endif
            drawOneCLs(TString::Format("acls_%s", "hzz4l"))

        
#if 0       #//---- TOYS -----
            drawOneCLs(TString::Format("cls_%s",  chann[i]))
            drawSMCLs(TString::Format("smcls_%s", chann[i]))
            drawMethods(TString::Format("cls_%s", chann[i]), /*include_pla=*/True)
            #//drawSMCLs(TString::Format("smcls_%s", chann[i]), True)
            #////drawOnePVal(TString::Format("pvala_%s", chann[i]), "Local p-value", /*toys=*/True)
            #//if (gFile.Get(Form("bayes_%s_obs", chann[i]))) drawMethods(TString::Format("acls_%s", chann[i]), /*include_pla=*/False)
#endif
	    #//drawOnePVal(TString::Format("pvala_%s", chann[i]), "Local p-value", /*toys=*/True)
            #//return
#if 0       #// P-values with bands
            if (i == 0):
                drawOnePValBand("pvala_comb")
                drawOnePValBand("pval_comb")
                drawOnePValBand("pvala_comb","pval_comb")
                drawOnePValBand("pval_comb","pvala_comb")
            }
#endif

#if 0       #// P.R. PLOTS
            if (i == 0):
                globalPrefix =  globalPrefix0+"/lte_"+prefixes[version] lep_excluded = True; tev_excluded = True; 
                drawOneCLs(TString::Format("cls_%s", chann[i]))
                globalPrefix =  globalPrefix0+"/ltc_"+prefixes[version] lep_excluded = True; tev_excluded = True; cms_excluded = True;
                drawOneCLs(TString::Format("cls_%s", chann[i]))
                globalPrefix =  globalPrefix0+"/ltl_"+prefixes[version] lep_excluded = True; tev_excluded = False; cms_excluded = False; Draw_TEV = True; 
                drawOneCLs(TString::Format("cls_%s", chann[i]))
                globalPrefix =  globalPrefix0+"/"+prefixes[version] lep_excluded = False; tev_excluded = False; Draw_TEV = False; cms_excluded = False;
            }
#endif
#if 0       #// NOW VS PAPER, COOL
            if (which == 0 and i == 0):
                drawTwoCLs(TString::Format("cls_%s", chann[i]),  TString::Format("cls_%s_paper", chann[i]), "Previous", "_hist", /*color=*/59, /*doObs2=*/False)
            }
#endif

#if 0       #// NOW VS PAPER
            if (which == 0):
                herePref = globalPrefix
                globalPrefix =  globalPrefix0+"/history/"+prefixes[version]
                channLP = chann[i]
                if (gFile.Get(TString::Format("cls_%s_obs", chann[i])) and gFile.Get(TString::Format("cls_%s_paper_obs", channLP.Data()))):
                    std::cout << "Doing 1-1 comparison of CLs plots for " << chann[i] << std::endl
                    drawTwoCLs(TString::Format("cls_%s", chann[i]),  TString::Format("cls_%s_paper", channLP.Data()), "HIG-11", "", /*color=*/59)
                } else if (gFile.Get(TString::Format("acls_%s_obs", chann[i])) and gFile.Get(TString::Format("acls_%s_paper_obs", channLP.Data()))):
                    std::cout << "Doing 1-1 comparison of Asym CLs plots for " << chann[i] << std::endl
                    drawTwoCLs(TString::Format("acls_%s", chann[i]), TString::Format("acls_%s_paper", channLP.Data()), "HIG-11", "", /*color=*/59)
                }
                if (gFile.Get(TString::Format("smcls_%s_obs", chann[i])) and gFile.Get(TString::Format("smcls_%s_paper_obs", channLP.Data()))):
                    std::cout << "Doing 1-1 comparison of SM CLs plots for " << chann[i] << std::endl
                    drawTwoSMCLs(TString::Format("smcls_%s", chann[i]),  TString::Format("smcls_%s_paper", channLP.Data()), "HIG-11", "", /*color=*/59)
                }
                globalPrefix = herePref
        }
#endif

        } #// loop over plots
    } #// loop over versions

#if 0   #// TIMING 
        globalPrefix = globalPrefix0+"/timing/"
        drawOnePlot(TString::Format("timecls_%s", chann[i]), "Time per CLs toy [s]")
#endif

#if 0 #// Interpolation stuff
    globalPrefix =  globalPrefix0+"/interpolation/"
    #//findCrossings("smcls_comb_obs", "low", 0.001, 120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_obs", "low99", 0.01,  120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_obs", "low95", 0.05,  120, 135, "CL_{S} of SM Higgs boson hypothesis")
    #//findCrossings("smcls_comb_obs", "high", 0.001, 120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_obs", "high99", 0.01,  450, 600, "CL_{S} of SM Higgs boson hypothesis")
    #//findCrossings("smcls_comb_obs", "high95", 0.05,  500, 600, "CL_{S} of SM Higgs boson hypothesis")
    #//findCrossings("smcls_comb_median", "low", 0.001, 120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_median", "low99", 0.01, 120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_median", "low95", 0.05, 110, 125, "CL_{S} of SM Higgs boson hypothesis")
    #//findCrossings("smcls_comb_median", "high", 0.001, 120, 135, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_median", "high99", 0.01, 400, 550, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("smcls_comb_median", "high95", 0.05, 500, 600, "CL_{S} of SM Higgs boson hypothesis")
    findCrossings("cls_comb_obs",      "low95", 1.0,  120, 135, "95% CL limit on #sigma/#sigma_{SM}")
    findCrossings("cls_comb_median",   "low95", 1.0,  110, 125, "95% CL limit on #sigma/#sigma_{SM}")
    findCrossings("cls_comb_obs",      "low95", 1.0,  120, 135, "95% CL limit on #sigma/#sigma_{SM}")
    findCrossings("cls_comb_median",   "low95", 1.0,  110, 125, "95% CL limit on #sigma/#sigma_{SM}")
    return
#endif


#if 0
    #/// ASYMPTOTICS VS TOYS
    rectangleCanvas() 
    globalPrefix =  globalPrefix0+"/asymptotic-valid/"
    for (int i = 0 i < nchann; ++i):
        drawCompLimit( TString::Format("acls_%s_obs",       chann[i]), TString::Format("cls_%s_obs",       chann[i]), "Asymp.", "Toys")
        drawCompLimit( TString::Format("acls_%s_median",    chann[i]), TString::Format("cls_%s_median",    chann[i]), "Asymp.", "Toys")
        drawCompLimit( TString::Format("acls_%s_median_95", chann[i]), TString::Format("cls_%s_median_95", chann[i]), "Asymp.", "Toys")
    }
#endif
#if 0
    #// ROOSTATS VS LANDS
    globalPrefix =  globalPrefix0+"/validation/"
    for (int i = 0 i < 1; ++i):
        drawCompLimit( TString::Format("acls_%s_obs",  chann[i]), TString::Format("acls_%s_lands_obs",  chann[i]), "RooStats", "LandS", "Asym CL_{S} obs. limit")
        drawCompLimit( TString::Format("acls_%s_median",  chann[i]), TString::Format("acls_%s_lands_median",  chann[i]), "RooStats", "LandS", "Asym CL_{S} exp. limit")
        drawCompPVal(  TString::Format("pvala_%s_obs",    chann[i]), TString::Format("pvala_%s_lands_obs", chann[i]), "RooStats", "LandS", "Local p-value obs.")
    }
#endif

#if 0
    globalPrefix =  globalPrefix0+"/private/"
    #//drawCompPVal("pvala_comb_asimov", "pval_comb_median",  "Asimov", "Med Toys", "Local p-value exp", 1e-8) 
    #//drawCompPVal("pvala_comb_asimov", "pvalh_comb_median", "Asimov", "Med Asymp", "Local p-value exp", 1e-8) 
    #//drawCompPVal("pvalh_comb_median", "pval_comb_median",  "Med Asymp", "Med Toys", "Local p-value exp", 1e-8) 
    #//drawCompPVal("pvala_comb_obs", "pvala_comb_lands_obs", "RooStats", "LandS", "Local p-value obs.")
    drawOnePValBand("pvala_comb")
    drawOnePValBand("pval_comb")
    drawOnePValBand("pvala_comb","pval_comb")
    drawOnePValBand("pval_comb","pvala_comb")
#endif


#if 0
    globalPrefix =  globalPrefix0+"/pvtests/"
    const int npvflav = 6
    *pvflav[npvflav] = { "M2M", "S", "SZ3", "BZ1", "BZ", "TQ0" }
    for (int i = 0 i < npvflav; ++i):
        std::cout << "Compare with " << Form("pvala_%s_comb_obs", pvflav[i]) << std::endl
        drawCompPVal(Form("pvala_%s_comb_obs", pvflav[i]), "pvala_comb_obs",   pvflav[i], "Median", "Local p-value") 
        #//drawCompPVal(Form("pvala_%s_comb_obs", pvflav[i]), "pval_comb_obs",    pvflav[i], "Toys", "Local p-value") 
        #//drawCompPVal(Form("pvala_%s_comb_obs", pvflav[i]), "pvala_BZ_comb_obs", pvflav[i], "BZ", "Local p-value") 
    }
#endif

#if 0
    #// EVENT-BY-EVENT
    globalPrefix =  globalPrefix0+"/other/"
    drawCompPVal( "pvala_comb_obs", "pvala_combe_obs", "Default", "Ev-by-ev.", "Local p-value obs."),
#endif

}

IndentationError: unexpected indent (<ipython-input-1-6efc9db071d9>, line 3)

In [23]:
#masses = loadMasses()
##print masses
#for m in masses:
#    print m
# THE END